In [39]:
import reset
from pathlib import Path
import numpy as np
import scipy as sp
import tifffile
import math
import matplotlib.pyplot as plt

In [40]:
# Assign variables from reset
animal_loc = reset.animal_loc
animal_id = reset.animal_id
hemi = reset.hemi
divisor = reset.divisor
down_x = reset.down_x
down_y = reset.down_y
grid = reset.grid
frame_len = reset.frame_len
max_pos = reset.max_pos

In [41]:
# Create list of locations as strings
pos_list = [str(n+1) for n in range(max_pos)]

# Define paths for input and output images
image_loc = animal_loc / "xmerged" / hemi
save_loc = animal_loc / "ymerged" / hemi

# Ensure save location exists
save_loc.mkdir(parents=True, exist_ok=True)

image_path_list = [f"{image_loc}/{animal_id}_{hemi}_{pos}.tif" for pos in pos_list]

In [42]:
def find_brightest_frame(path_num1, path_num2, path_num3):
    global max_frame
    """
    Find the brightest frame across three image sequences.
    Returns the index of the brightest frame and the average brightness for each image.
    """
    average_brightness = []
    for path_num in [path_num1, path_num2, path_num3]:
        brightness_list = []
        for fn in range(frame_len):
            im = tifffile.imread(image_path_list[path_num - 1])
            im_f = im[fn, :, :]
            brightness_list.append(np.average(im_f))
        
        max_brightness = max(brightness_list)
        max_frame_index = np.argmax(brightness_list)
        average_brightness.append((max_brightness, max_frame_index))

    # Determine the maximum brightness and corresponding frame
    max_brightness_values, max_frame_indices = zip(*average_brightness)
    max_frame = max_frame_indices[np.argmax(max_brightness_values)]
    
    print(f"Brightest frame across sections: {max_frame}")
    return max_frame

In [43]:
def y_connector(path_num1, path_num2, rect_y, rect_y_move, rect_y_start):
    global start_y
    # Load and extract frames from both images
    im1 = tifffile.imread(image_path_list[path_num1 - 1])
    im1_f = im1[max_frame, :, :]
    del im1

    im2 = tifffile.imread(image_path_list[path_num2 - 1])
    im2_f = im2[max_frame, :, :]
    del im2

    # Calculate correlations over specified range
    cor = [
        np.corrcoef(
            im1_f[:rect_y, :].ravel(),
            im2_f[i:i + rect_y, :].ravel()
        )[0, 1]
        for i in range(rect_y_start, rect_y_start + rect_y_move)
    ]

    start_y = np.argmax(cor) + rect_y

# Main function to align three sections vertically
def nine_sections(f1, f2, f3, frame_name, rect_y, rect_y_move, rect_y_start):
    global target1_start_y, target2_start_y

    y_connector(f1, f2, rect_y, rect_y_move, rect_y_start)
    target1_start_y = start_y
    y_connector(f2, f3, rect_y, rect_y_move, rect_y_start)
    target2_start_y = start_y

    print("First section target frame and start y:", max_frame, target1_start_y)
    print("Second section target frame and start y:", max_frame, target2_start_y)

    # Load images outside the loop to avoid repeated loading
    im1 = tifffile.imread(image_path_list[f1 - 1])
    im2 = tifffile.imread(image_path_list[f2 - 1])
    im3 = tifffile.imread(image_path_list[f3 - 1])

    width1 = im1.shape[2]
    width2 = im2.shape[2]
    width3 = im3.shape[2]

    # Align frames for all frames in `frame_len`
    for fn in range(frame_len):
        frame_name[fn, :height, :width1] = im1[fn,:,:]
        frame_name[fn, height:2 * height - target1_start_y, :width2] = im2[fn, target1_start_y:, :]
        frame_name[fn, 2 * height - target1_start_y:3 * height - target1_start_y - target2_start_y, :width3] = im3[fn, target2_start_y:, :]

    del im1, im2, im3

In [ ]:
#Adjust as deeded
rect_y = 20
rect_y_move = 200
rect_y_start = down_y - rect_y - rect_y_move

In [ ]:
height = down_y

frame_height = 3 * down_y
frame_width = 3 * down_x

# Initialize frames for top, middle, and bottom sections
frame = np.zeros((frame_len, frame_height, frame_width))

In [47]:
find_brightest_frame(1, 2, 3)
# Run alignment on specified sections
nine_sections(1, 2, 3, frame, rect_y, rect_y_move, rect_y_start)
# Save the aligned image
save_path = save_loc / f"{animal_id}_{hemi}_1.tif"
tifffile.imwrite(save_path, frame)
print(f"Aligned image saved to {save_path}")

Brightest frame across sections: 21
First section target frame and start y: 21 219
Second section target frame and start y: 21 153
Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/ymerged/l/rbak006_l_1.tif


In [48]:
find_brightest_frame(4, 5, 6)
# Run alignment on specified sections
nine_sections(4, 5, 6, frame, rect_y, rect_y_move, rect_y_start)
# Save the aligned image
save_path = save_loc / f"{animal_id}_{hemi}_2.tif"
tifffile.imwrite(save_path, frame)
print(f"Aligned image saved to {save_path}")

Brightest frame across sections: 59
First section target frame and start y: 59 100
Second section target frame and start y: 59 138
Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/ymerged/l/rbak006_l_2.tif
